In [34]:
import json
import time 

from kafka import KafkaProducer

import os


In [42]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [45]:
#question 3
producer.bootstrap_connected()

True

In [9]:
#question 4
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()

print(f'Sending took {(t1 - t0):.2f} seconds')

producer.flush()

t2 = time.time()
print(f'Flushing took {(t2 - t1):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
Sending took 0.50 seconds
Flushing took 0.00 seconds


In [17]:
import pandas as pd

In [21]:
#df_green_pd = pd.read_csv("green_tripdata_2019-10.csv")
ruta_archivo = "green_tripdata_2019-10.csv"

cols = [
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]

df_green_pd = pd.read_csv(ruta_archivo, usecols=cols)

In [27]:
df_green_pd.shape[0]

476386

In [25]:
cont = 0

for row in df_green_pd.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}    
    print(row_dict)

    if cont > 10:
        break

    cont+=1

{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': 112, 'DOLocationID': 196, 'passenger_count': 1.0, 'trip_distance': 5.88, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:18:11', 'lpep_dropoff_datetime': '2019-10-01 00:22:38', 'PULocationID': 43, 'DOLocationID': 263, 'passenger_count': 1.0, 'trip_distance': 0.8, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:09:31', 'lpep_dropoff_datetime': '2019-10-01 00:24:47', 'PULocationID': 255, 'DOLocationID': 228, 'passenger_count': 2.0, 'trip_distance': 7.5, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:37:40', 'lpep_dropoff_datetime': '2019-10-01 00:41:49', 'PULocationID': 181, 'DOLocationID': 181, 'passenger_count': 1.0, 'trip_distance': 0.9, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:08:13', 'lpep_dropoff_datetime': '2019-10-01 00:17:56', 'PULocationID': 97, 'DOLocationID': 188, 'passenger_count': 1.0, 'trip_distance': 2.52, 'tip

In [28]:
#question 5
t0 = time.time()

topic_name = 'green-trips'

for row in df_green_pd.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)

t1 = time.time()

print(f'Sending took {(t1 - t0):.2f} seconds')

producer.flush()

t2 = time.time()
print(f'Flushing took {(t2 - t1):.2f} seconds')

Sending took 65.48 seconds
Flushing took 11.06 seconds


In [29]:
print("Final time: "+str(65.48+11.06))

Final time: 76.54


In [53]:
pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

print(kafka_jar_package)

org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1


In [57]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

In [59]:
print("Spark Kafka version:", spark.conf.get("spark.jars.packages"))

Spark Kafka version: org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1


In [61]:
#from kafka import KafkaProducer

print(producer.bootstrap_connected())

green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

#.option("subscribe", "green-trips") \

False


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [14]:
df_green = spark.read.option("header", "true").csv('green_tripdata_2019-10.csv')